In [1]:
import numpy as np
import adi
import matplotlib
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'adi'

In [35]:
sdr = adi.Pluto()
sdr.rx_lo = int(2.4405e9)
sdr.rx_rf_bandwidth = int(56e6)
sdr.sample_rate = int(40e6)
sdr.rx_buffer_size = 1024

In [36]:
sdr

Pluto(uri="") object "PlutoSDR" with following key properties:

rx_lo:                   2440.5       MHz, Carrier frequency of RX path
rx_hardwaregain_chan0    53           dB, Gain applied to RX path. Only applicable when gain_control_mode is set to 'manual'
rx_rf_bandwidth:         56.0         MHz, Bandwidth of front-end analog filter of RX path
gain_control_mode_chan0: slow_attack  Receive path AGC Options: slow_attack, fast_attack, manual

tx_lo:                   2450.0       MHz, Carrier frequency of TX path
tx_hardwaregain_chan0:   -10          dB, Attenuation applied to TX path
tx_rf_bandwidth:         18.0         MHz, Bandwidth of front-end analog filter of TX path
tx_cyclic_buffer:        0            Toggles cyclic buffer

filter:                  [128.0, 2.0, 128.0, 2.0] FIR filter file
sample_rate:             40.0         MSPS, Sample rate RX and TX paths
loopback:                0            0=Disabled, 1=Digital, 2=RF


In [37]:
print("RX Buffer Size", sdr.rx_buffer_size)
print("RX Gain", sdr.rx_hardwaregain_chan0)
print("TX Gain", sdr.tx_hardwaregain_chan0)

RX Buffer Size 1024
RX Gain 30
TX Gain -10


In [35]:
def FFT(samples, buf, samprate, lo):
    fft = np.fft.fftshift(np.fft.fft(samples)) / buf
    freqLabels = np.fft.fftshift(np.fft.fftfreq(buf, 1/samprate)) + lo
    fft_mag_dB = 10*np.log10(np.abs(fft))
#     fft_mag = np.abs(fft)
    return fft_mag_dB, freqLabels

def MA(data):
    N = 10
    w = np.ones(N) / N
    ma = np.convolve(data, w, mode='same')
    return(ma)

def findSigPts(boolarray):
    a = boolarray^np.concatenate(([False],boolarray[:-1]))
    return(abs(a))

def findBWs(sigpts):
    freqs = np.zeros([len(sigpts)-1,2])
    freqs[:,0] = sigpts[1:]
    freqs[:,1] = sigpts[:-1]
    BWs = (sigpts[1:]-sigpts[:-1])/1e6
    return BWs, freqs

def findSigSec(boolarray):
    SigSections = []
    for i in range(1,len(boolarray)):
        if boolarray[i-1]:
            if i == 1:
                SigSections.append(1)
            if not boolarray[i]:
                SigSections.append(0)
        elif not boolarray[i-1]:
            if boolarray[i]:
                SigSections.append(1)

    return np.array(SigSections[:-1]).astype(np.bool)